In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pickle
from scipy.spatial import distance

def get_centroids(track, n_keypoints):

    centroids = []

    bodyparts = np.array([x for x in track.columns.get_level_values(1)])
    xpositions = pd.DataFrame(np.zeros((len(track),n_keypoints)), columns=np.unique(bodyparts))
    for i in range(n_keypoints):
        xpositions[np.unique(bodyparts)[i]] = (track[track.columns.get_level_values(0)[0], np.unique(bodyparts)[i], 'x'])
        
    median_xposition = np.median(xpositions, axis=1) #xpos

    ypositions = pd.DataFrame(np.zeros((len(track),n_keypoints)), columns=np.unique(bodyparts))
    for i in range(n_keypoints):
        ypositions[np.unique(bodyparts)[i]] = (track[track.columns.get_level_values(0)[0], np.unique(bodyparts)[i], 'y'])
        
    median_yposition = np.median(ypositions, axis=1) #ypos

    for j in range(len(median_xposition)):
        centroids.append([median_xposition[j], median_yposition[j]])

    return centroids

def calculate_velocity(x, sf, pix_to_cm):
    desplazamiento = [abs(distance.euclidean(x,y)/pix_to_cm) for x, y in zip(x[1:], x[:-1])]
    dt = 1 / sf
    v = [i/dt for i in desplazamiento]
    return v


folder =  r'C:\Users\Compras\Desktop\videos_DLC'
os.chdir(folder)
files = sorted(glob.glob('*clean.h5'))
print('hay ' + str(len(files)) + ' archivos')

def calc_time_moving(files):
    movement = 0
    total = 0 
    for file in files:
            track = pd.read_hdf(file)

            centroids = get_centroids(track, n_keypoints=5)
            velocity = calculate_velocity(centroids, sf=50, pix_to_cm=10)

            idx_movement = [i for i,x in enumerate(velocity) if x > 5] #filtramos frames con mas de 10cm/s de v

            movement += len(idx_movement)
            
            total += len(track)

    time_moving = (movement * 100 / total)
    print(time_moving)


hay 13 archivos


In [95]:
files[7:]

['video_tank2_2024-01-25T09_12_41DLC_resnet50_GymnotusDec19shuffle1_200000_clean.h5',
 'video_tank2_2024-01-25T09_32_41DLC_resnet50_GymnotusDec19shuffle1_200000_clean.h5',
 'video_tank2_2024-01-25T09_52_41DLC_resnet50_GymnotusDec19shuffle1_200000_clean.h5',
 'video_tank2_2024-01-25T10_12_42DLC_resnet50_GymnotusDec19shuffle1_200000_clean.h5',
 'video_tank2_2024-01-25T10_32_42DLC_resnet50_GymnotusDec19shuffle1_200000_clean.h5',
 'video_tank2_2024-01-25T10_52_42DLC_resnet50_GymnotusDec19shuffle1_200000_clean.h5']

In [96]:
files[48:]

[]

In [97]:
calc_time_moving(files)
calc_time_moving(files[:7])
calc_time_moving(files[7:])
#calc_time_moving(files[12:36])
#calc_time_moving(files[36:42])
#calc_time_moving(files[42:48])
#calc_time_moving(files[48:])

0.0
0.0
0.0
